In [1]:
import bigframes
import bigframes.pandas as bpd
import bigframes.streaming as bfs
import inspect

In [2]:
bigframes.options._bigquery_options.project = "bigframes-load-testing"
job_id_prefix = "test_streaming_"

/usr/local/google/home/garrettwu/src/bigframes/bigframes/pandas/__init__.py:866: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return global_session.get_global_session()


### Create, select, filter and preview

In [3]:
sdf = bpd.read_gbq_table_streaming("birds.penguins_bigtable_streaming")

In [4]:
sdf = sdf[["species", "island", "body_mass_g"]]
sdf = sdf[sdf["body_mass_g"] < 4000]
sdf = sdf.rename(columns={"island": "rowkey"})
print(type(sdf))
sdf

<class 'bigframes.streaming.StreamingDataFrame'>


,species,rowkey,body_mass_g
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,3875
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,2900
5,Adelie Penguin (Pygoscelis adeliae),Biscoe,3725
6,Adelie Penguin (Pygoscelis adeliae),Dream,2975
11,Adelie Penguin (Pygoscelis adeliae),Torgersen,3050
14,Chinstrap penguin (Pygoscelis antarctica),Dream,2700
15,Adelie Penguin (Pygoscelis adeliae),Dream,3900
17,Adelie Penguin (Pygoscelis adeliae),Biscoe,3825
21,Chinstrap penguin (Pygoscelis antarctica),Dream,3775
22,Adelie Penguin (Pygoscelis adeliae),Dream,3350


### BigTable

In [6]:
job = sdf.to_bigtable(instance="streaming-testing-instance",
    table="garrettwu-no-col-family",
    service_account_email="streaming-testing-admin@bigframes-load-testing.iam.gserviceaccount.com",
    app_profile=None,
    truncate=True,
    overwrite=True,
    auto_create_column_families=True,
    bigtable_options={},
    job_id=None,
    job_id_prefix=job_id_prefix,)

/usr/local/google/home/garrettwu/src/bigframes/bigframes/streaming/__init__.py:320: PreviewWarning: The bigframes.streaming module is a preview feature, and subject to change.
  warnings.warn(


In [7]:
print(job.running())
print(job.error_result)

True
None


In [8]:
job.cancel()

True

### PubSub

In [9]:
sdf = sdf[["rowkey"]]

In [10]:
job = sdf.to_pubsub(
        topic="penguins",
        service_account_email="streaming-testing@bigframes-load-testing.iam.gserviceaccount.com",
        job_id=None,
        job_id_prefix=job_id_prefix,
    )

/usr/local/google/home/garrettwu/src/bigframes/bigframes/streaming/__init__.py:435: PreviewWarning: The bigframes.streaming module is a preview feature, and subject to change.
  warnings.warn(


In [11]:
print(job.running())
print(job.error_result)

True
None


In [12]:
job.cancel()

True